## IMPORTS

In [ ]:
import networkx as nx 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp 
import random
from scipy import sparse
import scipy.io
import seaborn as sb
import math
import copy
import sklearn
from sklearn.model_selection import ParameterGrid

# 1. Influenza H1N1 2009 Pandemic in Sweden

## PROBLEM 1.1

In [ ]:
n_nodes = 500

#links activation rate
beta = 0.3
#spontaneous mutation rate
rho = 0.7
#degree of each node
k = 4

G = nx.cycle_graph(n_nodes)

for n in range(n_nodes):
    G.add_edge(n,((n+2) % n_nodes))
    G.add_edge(n,((n-2) % n_nodes))

pos = nx.circular_layout(G)

nx.draw(G,pos,with_labels=True)

In [ ]:
def insert_infected_2(G):
    infected = np.random.choice(np.arange(500), size=10, replace=False)
    return infected

In [ ]:
def num_infected(G, el, G_n):
    list_neighbors = list(G.neighbors(el))
    list_neighbors.sort()
    neig_value = [G_n[n] for n in list_neighbors]
    num_infected = neig_value.count(1)
    return num_infected


In [ ]:
c = 0
list_infected = [[] for i in range(15)]
list_susceptible = [[] for i in range(15)]
list_recovered = [[] for i in range(15)]
list_newly_infected = [[] for i in range(15)]

for c in range(100):
    #create a graph with 500 nodes of which 10 are infected
    G_n = [0]*500
    for el in insert_infected_2(G):
        G_n[el] = 1
        
    #simulate for 15 weeks the pandemic
    for i in range(15):
        num_newly_infected = 0
        list_update_1 = []
        list_update_2 = []
        #iterate over all nodes
        for j in range(n_nodes):
            p = np.random.random()
            if G_n[j] == 0:
                w = num_infected(G,j, G_n)
                if w > 0:
                    if p< 1-((1-beta)**w):
                        list_update_1.append(j)

            elif G_n[j] == 1:
                if p < rho:
                    list_update_2.append(j)

        #at the end of the week i update the states susceptible-infected and infected-recovered
        for el in list_update_1:
            G_n[el] = 1
            num_newly_infected += 1
        for el in list_update_2:
            G_n[el] = 2
        
        #append the lists
        list_susceptible[i].append(G_n.count(0))
        list_infected[i].append(G_n.count(1))
        list_recovered[i].append(G_n.count(2))
        list_newly_infected[i].append(num_newly_infected)
       
    

In [ ]:
#plot how many people became infected each week:
fig, ax = plt.subplots(figsize=(10,6))
num_weeks = [i for i in range(0, 15)]
avg_new_infected = []
avg_susceptible = []
avg_infected = []
avg_recovered = []
for i in range (15):
    avg_new_infected.append(sum(list_newly_infected[i])/len(list_newly_infected[i]))
    avg_susceptible.append(sum(list_susceptible[i])/len(list_susceptible[i]))
    avg_infected.append(sum(list_infected[i])/len(list_infected[i]))
    avg_recovered.append(sum(list_recovered[i])/len(list_recovered[i]))

ax.plot(num_weeks, avg_new_infected)
plt.yticks(np.arange(0, 20, 2, dtype=int))
plt.xticks(np.arange(0, 15, 1, dtype=int))
ax.grid()
plt.savefig('problem1.1.png', format='PNG')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))

ax.plot(num_weeks, avg_susceptible, label = "susceptible")
ax.plot(num_weeks, avg_infected, label = "infected")
ax.plot(num_weeks, avg_recovered, label = "recovered")
plt.yticks(np.arange(0, 500, 50, dtype=int))
ax.grid()
ax.legend()
plt.savefig('problem1.1_2.png', format='PNG')
plt.show()

## PROBLEM 1.2

In [ ]:
#function that returns the degree of the new nodes added
def degree_new_nodes(k, n_times):
    c = 0
    if k%2 == 0:
        c = k/2
    else:
        if n_times%2 ==0:
            c = math.trunc(k/2)
        else:
            c = math.ceil(k/2)
    return c


In [ ]:

def build_random_graph(n_nodes, degree):

    k = degree
    #initialize the complete graph with k+1 nodes
    G_2 = nx.complete_graph(k+1)
    W = nx.adjacency_matrix(G_2)
    list_degree_nodes = [k for el in G_2.nodes]

    num_nodes = len(G_2.nodes)
    counter_num_times = 0

    while num_nodes< n_nodes:
        
        G_2.add_node(num_nodes)
        c = int(degree_new_nodes(k, counter_num_times))
        #pick c number of tail nodes
        tail_nodes = np.random.choice(len(list_degree_nodes), size= c,  p= [list_degree_nodes[i]/sum(list_degree_nodes) for i in range(len(list_degree_nodes))], replace=False)
        #update the degree of the tail nodes and add the edge from them to the new node
        for el in tail_nodes:
            list_degree_nodes[el] += 1
            G_2.add_edge(el, num_nodes)

        #update step
        num_nodes += 1
        counter_num_times += 1
        list_degree_nodes.append(c)
    
    avg_degree = sum(list_degree_nodes)/len(list_degree_nodes)

    return G_2, avg_degree

## PROBLEM 2

In [ ]:
#links activation rate
beta = 0.3
#spontaneous mutation rate
rho = 0.7
#degree of each node
k = 6

G, avg_degree = build_random_graph(500, 6)
nx.draw(G)
print(avg_degree)
print(len(G.nodes))

In [ ]:

#generate a preferential attachment random graph of average degree 6
n_nodes = 500
G, avg_degree = build_random_graph(n_nodes, 6)
list_infected = [[] for i in range(15)]
list_susceptible = [[] for i in range(15)]
list_recovered = [[] for i in range(15)]
list_newly_infected = [[] for i in range(15)]

for c in range(100):
    #create a graph with 500 nodes of which 10 are infected
    G_n = [0]*500
    for el in insert_infected_2(G):
        G_n[el] = 1
    #simulate for 15 weeks the pandemic
    for i in range(15):
        num_newly_infected = 0
        list_update_1 = []
        list_update_2 = []
        #iterate over all nodes
        for j in range(n_nodes):
            p = np.random.random()
            if G_n[j] == 0:
                w = num_infected(G,j, G_n)
                if w > 0:
                    if p< 1-((1-beta)**w):
                        list_update_1.append(j)

            elif G_n[j] == 1:
                if p < rho:
                    list_update_2.append(j)

        #at the end of the week i update the states susceptible-infected and infected-recovered
        for el in list_update_1:
            G_n[el] = 1
            num_newly_infected += 1
        for el in list_update_2:
            G_n[el] = 2
        
        #append the lists
        list_susceptible[i].append(G_n.count(0))
        list_infected[i].append(G_n.count(1))
        list_recovered[i].append(G_n.count(2))
        list_newly_infected[i].append(num_newly_infected)
       

In [ ]:
#plot how many people became infected each week:
fig, ax = plt.subplots(figsize=(10,6))
num_weeks = [i for i in range(0, 15)]
avg_new_infected = []
avg_susceptible = []
avg_infected = []
avg_recovered = []

for i in range (15):
    if len(list_newly_infected[i]) == 0:
        den = 1
    else:
        den = len(list_newly_infected[i])

    avg_new_infected.append(sum(list_newly_infected[i])/den)
    avg_susceptible.append(sum(list_susceptible[i])/len(list_susceptible[i]))
    avg_infected.append(sum(list_infected[i])/len(list_infected[i]))
    avg_recovered.append(sum(list_recovered[i])/len(list_recovered[i]))

ax.plot(num_weeks, avg_new_infected)
plt.yticks(np.arange(0, 120, 10, dtype=int))
plt.xticks(np.arange(0, 15, 1, dtype=int))
ax.grid()
plt.savefig('problem1.2.png', format='PNG')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))

ax.plot(num_weeks, avg_susceptible, label = "susceptible")
ax.plot(num_weeks, avg_infected, label = "infected")
ax.plot(num_weeks, avg_recovered, label = "recovered")
plt.yticks(np.arange(0, 500, 50, dtype=int))
ax.grid()
ax.legend()
plt.savefig('problem1.2_2.png', format='PNG')
plt.show()

## PROBLEM 3

In [ ]:
#function that given the week returns the the number of people that should be vaxinated during the week
def num_vaccinated(index_week, tot):
    Vacc = [0, 5, 15, 25, 35, 45, 55, 60, 60, 60, 60, 60, 60, 60, 60]
    num_nodes = 0
    for i, el in enumerate(Vacc):
        arr = np.array(Vacc)
        if i ==index_week:
            if i == 0:
                num_nodes = 0
            else:
                num_nodes = int(np.round((el-arr[i-1])/100*tot))

    return num_nodes

In [ ]:
#check if the elements of list1 (to vaccinate) are not in list2 (already vaccinated), if there are replace them with others that are in list3 (with nodes not vaccinated)
def check_list (list1, list2, G):
    list_all = [el for el in G.nodes]
    list3 = list(set(list_all)-set(list2)-set(list1))
    for el in list1: 
        if el in list2:
            new_el = random.choice(list3)
            list1 = [new_el if x==el else x for x in list1]
            list2.append(new_el)
        else:
            list2.append(el)
            
    return list1, list2

In [ ]:
n_nodes = 500
#links activation rate
beta = 0.3
#spontaneous mutation rate
rho = 0.7
#degree of each node
k = 6

G, avg = build_random_graph(500, 6)
print(avg)
nx.draw(G)

In [ ]:
#simulate the pandemic with vaccination
c = 0
list_infected = [[] for i in range(15)]
list_susceptible = [[] for i in range(15)]
list_recovered = [[] for i in range(15)]
list_vaccinated_week = [[] for i in range(15)]
list_newly_infected = [[] for i in range(15)]
list_newly_vaccinated = [ [] for i in range(15)]

for c in range(100):
    #create a graph with 500 nodes, 10 of which are infected
    G_n = [0]*500
    for el in insert_infected_2(G):
        G_n[el] = 1
    check_vaccine = []

    #simulate for 15 weeks the pandemic
    for i in range(15):
        num_newly_infected = 0
        list_update_1 = []
        list_update_2 = []        
        #vaxinate a part of the population according to the week
        n_vaccinations = num_vaccinated(i, 500)
        #creo lista che vaccini random tot nodi
        vac = list(np.random.choice(n_nodes, size =  int(n_vaccinations), replace = False))
        #check that all the node to vaccinate are not already vaccinated
        list_2_vaccinate, check_vaccine = check_list(vac, check_vaccine, G)
        #update the state of vaccinated individuals "live" because the effects of vaccine taxes place immediately
        for el in list_2_vaccinate:
            if G_n[el]!=2:
                G_n[el] = 3 
        
        #iterate over all nodes
        for j in range(n_nodes):
            p = np.random.random()
            if G_n[j] == 0:
                w = num_infected(G,j, G_n)
                if p< 1-((1-beta)**w):
                    list_update_1.append(j)

            elif G_n[j] == 1:
                if p < rho:
                    list_update_2.append(j)

        #at the end of the week i update the states susceptible-infected and infected-recovered
        for el in list_update_1:
            G_n[el] = 1
            num_newly_infected += 1
        for el in list_update_2:
            G_n[el] = 2
        
        #append the lists
        list_susceptible[i].append(G_n.count(0))
        list_infected[i].append(G_n.count(1))
        list_recovered[i].append(G_n.count(2))
        list_vaccinated_week[i].append(len(check_vaccine))
        list_newly_infected[i].append(num_newly_infected)
        list_newly_vaccinated[i].append(len(list_2_vaccinate))
    

In [ ]:
#plot how many people became infected each week:
fig, ax = plt.subplots(figsize=(10,6))
num_weeks = [i for i in range(0, 15)]
avg_new_infected = []
avg_new_vaccinated = []
avg_susceptible = []
avg_infected = []
avg_recovered = []
avg_vaccinated = []
sum_check = []
for i in range (15):
    avg_new_infected.append(sum(list_newly_infected[i])/len(list_newly_infected[i]))
    if len(list_newly_vaccinated[i])==0:
        den = 1
    else:
        den = len(list_newly_vaccinated[i])
    avg_new_vaccinated.append(sum(list_newly_vaccinated[i])/den)
    avg_susceptible.append(sum(list_susceptible[i])/len(list_susceptible[i]))
    avg_infected.append(sum(list_infected[i])/len(list_infected[i]))
    avg_recovered.append(sum(list_recovered[i])/len(list_recovered[i]))
    avg_vaccinated.append(sum(list_vaccinated_week[i])/len(list_vaccinated_week[i]))
    sum_check.append(sum(list_susceptible[i]+list_infected[i]+list_recovered[i]))

ax.plot(num_weeks, avg_new_infected, label='new infected')
ax.plot(num_weeks, avg_new_vaccinated, label='new vaccinated')
plt.yticks(np.arange(0, 80, 5, dtype=int))
plt.xticks(np.arange(0, 15, 1, dtype=int))
ax.grid()
ax.legend()
plt.savefig('problem1.3.png', format='PNG')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))

ax.plot(num_weeks, avg_susceptible, label = "susceptible")
ax.plot(num_weeks, avg_infected, label = "infected")
ax.plot(num_weeks, avg_recovered, label = "recovered")
ax.plot(num_weeks, avg_vaccinated, label = "vaccinated")
plt.yticks(np.arange(0, 500, 50, dtype=int))
ax.grid()
ax.legend()
plt.savefig('problem1.3_2.png', format='PNG')
plt.show()

 ## PROBLEM 4

In [ ]:
#function that given the week returns the number of nodes to vaccinate
def num_nodes2vaccinate(index_week, tot):
    Vacc = [0, 5, 9, 16, 24, 32, 40, 47, 54, 59, 60, 60, 60, 60, 60, 60]
    for i, el in enumerate(Vacc):
        arr = np.array(Vacc)
        if i ==index_week:
            if i == 0:
                num_nodes = int(np.round(5/100*tot))
            else:
                num_nodes = int(np.round((el-arr[i-1])/100*tot))

    return num_nodes

In [ ]:
'''ALGORITHM FOR SEARCHING BETA, K AND RHO (k0 = 10, β0 = 0.3, ρ0 = 0.6 as an initial guess)'''

def simulate_pandemic_with_vaccination(n_nodes, k, beta, rho, n_infected_init):
    #build random graph
    G, avg_degree = build_random_graph(n_nodes, k)
    #lists for weekly updates
    list_infected = [[] for i in range(15)]
    list_susceptible = [[] for i in range(15)]
    list_recovered = [[] for i in range(15)]
    list_vaccinated_week = [[] for i in range(15)]
    list_newly_infected = [[] for i in range(15)]
    list_newly_vaccinated = [ [] for i in range(15)]
    #list for statistics:
    avg_new_infected = []
    avg_new_vaccinated = []
    avg_susceptible = []
    avg_infected = []
    avg_recovered = []
    avg_vaccinated = []
    #list for computing the error
    list_results = []
    
    for c in range(100):
        #create a graph with n_nodes, 10 of which are infected
        G_n = [0]*n_nodes
        infected = np.random.choice(n_nodes, size=n_infected_init, replace=False)
        for el in infected:
            G_n[el] = 1
        #list that check that you are not vaccinating a person more than one time
        check_vaccine = []

        #simulate for 15 weeks the pandemic
        for i in range(15):
            num_newly_infected = 0
            list_update_1 = []
            list_update_2 = []        
            #vaxinate a part of the population according to the new vaccination scheme
            n_vaccinations = num_nodes2vaccinate(i, n_nodes)
            #creo lista che vaccini random tot nodi
            vac = list(np.random.choice(n_nodes, size =  int(n_vaccinations), replace = False))
            #check that all the node to vaccinate are not already vaccinated
            list_2_vaccinate, list_vaccinated = check_list(vac, check_vaccine, G)
            #update check vaccine
            check_vaccine = list_vaccinated
            #update the state of vaccinated individuals live because the effects of vaccine taxes place immediately
            for el in list_2_vaccinate:
                if G_n[el] !=2:
                    G_n[el] = 3
            
            #iterate over all nodes
            for j in range(n_nodes):
                p = np.random.random()
                if G_n[j] == 0:
                    w = num_infected(G,j, G_n)
                    if w > 0:
                        if p< 1-((1-beta)**w):
                            list_update_1.append(j)

                elif G_n[j] == 1:
                    if p < rho:
                        list_update_2.append(j)

            #at the end of the week i update the states susceptible-infected and infected-recovered
            for el in list_update_1:
                G_n[el] = 1
                num_newly_infected += 1
            for el in list_update_2:
                G_n[el] = 2

            #append the lists
            list_susceptible[i].append(G_n.count(0))
            list_infected[i].append(G_n.count(1))
            list_recovered[i].append(G_n.count(2))
            list_vaccinated_week[i].append(len(check_vaccine))
            list_newly_infected[i].append(num_newly_infected)
            list_newly_vaccinated[i].append(n_vaccinations)
    
    #list results  will give the avg number of newly infected individuals for each week
    for i in range(15):
        if len(list_newly_infected[i])==0:
            den = 1
        else:
            den = len(list_newly_infected[i])

        list_results.append(sum(list_newly_infected[i])/den)

    
    return list_results, list_susceptible, list_infected, list_recovered, list_vaccinated_week, list_newly_infected, list_newly_vaccinated

In [ ]:
#define the error
def rmse(list_i):
    list_i0 = [1, 1, 3, 5, 9, 17, 32, 32, 17, 5, 2, 1, 0, 0, 0]
    sub_list_sqrt = [(x1-x2)**2 for (x1,x2) in zip(list_i, list_i0)]
    return math.sqrt(1/15* sum(sub_list_sqrt))

In [ ]:
initialk0 = 10
initialβ0 = 0.3
initialρ0 = 0.6
deltak0 = 1
deltaβ0 = 0.1
deltaρ0 = 0.1

params = {
    "k":[initialk0-deltak0, initialk0, initialk0+deltak0],
    "beta":[initialβ0-deltaβ0, initialβ0, initialβ0+deltaβ0],
    "rho":[initialρ0-deltaρ0, initialρ0, initialρ0+deltaρ0]
}

dict_rmse = {}


for config in ParameterGrid(params):
    list_results, list_susceptible, list_infected, list_recovered, list_vaccinated_week, list_newly_infected, list_newly_vaccinated = simulate_pandemic_with_vaccination(934, config['k'],config['beta'], config['rho'],1)
    error = rmse(list_results)
    dict_rmse[error] = config



In [ ]:
print(dict_rmse)
print(min(dict_rmse.keys()))
print(dict_rmse[min(dict_rmse.keys())])

In [ ]:
#run the simulation with the founded params and plot the results
list_results, list_susceptible, list_infected, list_recovered, list_vaccinated_week, list_newly_infected, list_newly_vaccinated = simulate_pandemic_with_vaccination(934, 9, 0.2, 0.7, 1)
fig, ax = plt.subplots(figsize=(10,6))
list_i0 = [1, 1, 3, 5, 9, 17, 32, 32, 17, 5, 2, 1, 0, 0, 0]
print(len(list_i0))
ax.plot(num_weeks, list_i0, label = 'true newly infected')
ax.plot(num_weeks, list_results, label='estimated newly infected')
ax.legend()
ax.grid()
plt.savefig('problem1.4.png', format='PNG')
ax.plot()

In [ ]:
avg_new_infected = []
avg_new_vaccinated = []
avg_susceptible = []
avg_infected = []
avg_recovered = []
avg_vaccinated = []
sum_check = []
for i in range (15):
    avg_new_infected.append(sum(list_newly_infected[i])/len(list_newly_infected[i]))
    if len(list_newly_vaccinated[i])==0:
        den = 1
    else:
        den = len(list_newly_vaccinated[i])
    avg_new_vaccinated.append(sum(list_newly_vaccinated[i])/den)
    avg_susceptible.append(sum(list_susceptible[i])/len(list_susceptible[i]))
    avg_infected.append(sum(list_infected[i])/len(list_infected[i]))
    avg_recovered.append(sum(list_recovered[i])/len(list_recovered[i]))
    avg_vaccinated.append(sum(list_vaccinated_week[i])/len(list_vaccinated_week[i]))
    sum_check.append(sum(list_susceptible[i]+list_infected[i]+list_recovered[i]))

fig, ax = plt.subplots(figsize=(10,6))
ax.plot(num_weeks, avg_susceptible, label = "susceptible")
ax.plot(num_weeks, avg_infected, label = "infected")
ax.plot(num_weeks, avg_recovered, label = "recovered")
ax.plot(num_weeks, avg_vaccinated, label = "vaccinated")
plt.yticks(np.arange(0, 950, 50, dtype=int))
ax.grid()
ax.legend()
plt.show()

In [ ]:
initialk0 = 10
initialβ0 = 0.3
initialρ0 = 0.6
deltak0 = 3
deltaβ0 = 0.05
deltaρ0 = 0.05

params = {
    "k":[initialk0-deltak0, initialk0, initialk0+deltak0],
    "beta":[initialβ0-deltaβ0, initialβ0, initialβ0+deltaβ0],
    "rho":[initialρ0-deltaρ0, initialρ0, initialρ0+deltaρ0]
}

dict_rmse = {}


for config in ParameterGrid(params):
    list_results, list_susceptible, list_infected, list_recovered, list_vaccinated_week, list_newly_infected, list_newly_vaccinated = simulate_pandemic_with_vaccination(934, config['k'],config['beta'], config['rho'],1)
    error = rmse(list_results)
    dict_rmse[error] = config



In [ ]:
print(dict_rmse[min(dict_rmse.keys())])
print(min(dict_rmse.keys()))

In [ ]:
initialk0 = 10
initialβ0 = 0.3
initialρ0 = 0.6
deltak0 = 3
deltaβ0 = 0.05
deltaρ0 = 0.05

params = {
    "k":[initialk0-deltak0, initialk0, initialk0+deltak0],
    "beta":[initialβ0-deltaβ0, initialβ0, initialβ0+deltaβ0],
    "rho":[initialρ0-deltaρ0, initialρ0, initialρ0+deltaρ0]
}

dict_rmse = {}


for config in ParameterGrid(params):
    list_results, list_susceptible, list_infected, list_recovered, list_vaccinated_week, list_newly_infected, list_newly_vaccinated = simulate_pandemic_with_vaccination(934, config['k'],config['beta'], config['rho'],1)
    error = rmse(list_results)
    dict_rmse[error] = config


print(dict_rmse[min(dict_rmse.keys())])
print(min(dict_rmse.keys()))

We keep fixed the delta of beta and rho and search for the optimal value of increasing the possibilities.


In [ ]:
initialβ0 = 0.3
initialρ0 = 0.6
deltaβ0 = 0.05
deltaρ0 = 0.05

params = {
    "k":[5, 6, 7, 8, 9, 10],
    "beta":[initialβ0-deltaβ0, initialβ0, initialβ0+deltaβ0],
    "rho":[initialρ0-deltaρ0, initialρ0, initialρ0+deltaρ0]
}

dict_rmse = {}


for config in ParameterGrid(params):
    list_results, list_susceptible, list_infected, list_recovered, list_vaccinated_week, list_newly_infected, list_newly_vaccinated = simulate_pandemic_with_vaccination(934, config['k'],config['beta'], config['rho'],1)
    error = rmse(list_results)
    dict_rmse[error] = config


print(dict_rmse[min(dict_rmse.keys())])
print(min(dict_rmse.keys()))

The combination above is the best that we can obtain, then we plot the result:

In [ ]:
list_results, list_susceptible, list_infected, list_recovered, list_vaccinated_week, list_newly_infected, list_newly_vaccinated = simulate_pandemic_with_vaccination(934, 6, 0.25, 0.55, 1)
fig, ax = plt.subplots(figsize=(10,6))
list_i0 = [1, 1, 3, 5, 9, 17, 32, 32, 17, 5, 2, 1, 0, 0, 0]
print(len(list_i0))
ax.plot(num_weeks, list_i0, label = 'true newly infected')
ax.plot(num_weeks, list_results, label='estimated newly infected')
ax.legend()
ax.grid()
plt.savefig('problem1.4_2.png', format='PNG')
ax.plot()

In [ ]:
avg_new_infected = []
avg_new_vaccinated = []
avg_susceptible = []
avg_infected = []
avg_recovered = []
avg_vaccinated = []
sum_check = []
for i in range (15):
    avg_new_infected.append(sum(list_newly_infected[i])/len(list_newly_infected[i]))
    if len(list_newly_vaccinated[i])==0:
        den = 1
    else:
        den = len(list_newly_vaccinated[i])
    avg_new_vaccinated.append(sum(list_newly_vaccinated[i])/den)
    avg_susceptible.append(sum(list_susceptible[i])/len(list_susceptible[i]))
    avg_infected.append(sum(list_infected[i])/len(list_infected[i]))
    avg_recovered.append(sum(list_recovered[i])/len(list_recovered[i]))
    avg_vaccinated.append(sum(list_vaccinated_week[i])/len(list_vaccinated_week[i]))
    sum_check.append(sum(list_susceptible[i]+list_infected[i]+list_recovered[i]))

fig, ax = plt.subplots(figsize=(10,6))

ax.plot(num_weeks, avg_susceptible, label = "susceptible")
ax.plot(num_weeks, avg_infected, label = "infected")
ax.plot(num_weeks, avg_recovered, label = "recovered")
ax.plot(num_weeks, avg_vaccinated, label = "vaccinated")
plt.yticks(np.arange(0, 950, 50, dtype=int))
ax.grid()
ax.legend()
plt.savefig('problem1.4_3.png', format='PNG')
plt.show()

## CHALLENGE

Try to find a better random graph (i.e. one that does not use preferential attachment) to
represent the network for the pandemic.

In [ ]:

'''THE FUNCTION IS THE SAME OF THE SIRV MODEL USED FOR H1N1 EPIDEMIC BUT WITH A NEWMANN-WATTS-STROGATZ GRAPH'''
def simulate_pandemic_with_vaccination_challenge(n_nodes,p, k, beta, rho, n_infected_init):
    #build a graph using Newman-Watts-Strogatz model
    G = nx.newman_watts_strogatz_graph(n_nodes, k, p)
    #lists for weekly updates
    list_infected = [[] for i in range(15)]
    list_susceptible = [[] for i in range(15)]
    list_recovered = [[] for i in range(15)]
    list_vaccinated_week = [[] for i in range(15)]
    list_newly_infected = [[] for i in range(15)]
    list_newly_vaccinated = [ [] for i in range(15)]
    #list for statistics:
    avg_new_infected = []
    avg_new_vaccinated = []
    avg_susceptible = []
    avg_infected = []
    avg_recovered = []
    avg_vaccinated = []
    #list for computing the error
    list_results = []
    
    for c in range(100):
        #create a graph with n_nodes, 10 of which are infected
        G_n = [0]*n_nodes
        infected = np.random.choice(n_nodes, size=n_infected_init, replace=False)
        for el in infected:
            G_n[el] = 1
        #list that check that you are not vaccinating a person more than one time
        check_vaccine = []

        #simulate for 15 weeks the pandemic
        for i in range(15):
            num_newly_infected = 0
            list_update_1 = []
            list_update_2 = []        
            #vaxinate a part of the population according to the new vaccination scheme
            n_vaccinations = num_nodes2vaccinate(i, n_nodes)
            #creo lista che vaccini random tot nodi
            vac = list(np.random.choice(n_nodes, size =  int(n_vaccinations), replace = False))
            #check that all the node to vaccinate are not already vaccinated
            list_2_vaccinate, list_vaccinated = check_list(vac, check_vaccine, G)
            #update check vaccine
            check_vaccine = list_vaccinated
            #update the state of vaccinated individuals live because the effects of vaccine taxes place immediately
            for el in list_2_vaccinate:
                if G_n[el] !=2:
                    G_n[el] = 3
            
            #iterate over all nodes
            for j in range(n_nodes):
                p = np.random.random()
                if G_n[j] == 0:
                    w = num_infected(G,j, G_n)
                    if w > 0:
                        if p< 1-((1-beta)**w):
                            list_update_1.append(j)

                elif G_n[j] == 1:
                    if p < rho:
                        list_update_2.append(j)

            #at the end of the week i update the states susceptible-infected and infected-recovered
            for el in list_update_1:
                G_n[el] = 1
                num_newly_infected += 1
            for el in list_update_2:
                G_n[el] = 2

            #append the lists
            list_susceptible[i].append(G_n.count(0))
            list_infected[i].append(G_n.count(1))
            list_recovered[i].append(G_n.count(2))
            list_vaccinated_week[i].append(len(check_vaccine))
            list_newly_infected[i].append(num_newly_infected)
            list_newly_vaccinated[i].append(n_vaccinations)
    
    #list results  will give the avg number of newly infected individuals for each week
    for i in range(15):
        if len(list_newly_infected[i])==0:
            den = 1
        else:
            den = len(list_newly_infected[i])

        list_results.append(sum(list_newly_infected[i])/den)

    
    return list_results, list_susceptible, list_infected, list_recovered, list_vaccinated_week, list_newly_infected, list_newly_vaccinated   

In [ ]:
initialk0 = 9
initialβ0 = 0.3
initialρ0 = 0.6
deltak0 = 3
deltaβ0 = 0.05
deltaρ0 = 0.05
initialp = 0.5
deltap = 0.1

params = {
    "k":[initialk0-deltak0, initialk0, initialk0+deltak0],
    "beta":[initialβ0-deltaβ0, initialβ0, initialβ0+deltaβ0],
    "rho":[initialρ0-deltaρ0, initialρ0, initialρ0+deltaρ0],
    "p":[initialp-deltap, initialp, initialp+deltap]
}

dict_rmse = {}


for config in ParameterGrid(params):
    list_results, list_susceptible, list_infected, list_recovered, list_vaccinated_week, list_newly_infected, list_newly_vaccinated = simulate_pandemic_with_vaccination_challenge(934,config['p'], config['k'],config['beta'], config['rho'],1)
    error = rmse(list_results)
    dict_rmse[error] = config


print(dict_rmse[min(dict_rmse.keys())])
print(min(dict_rmse.keys()))

In [ ]:
list_results, list_susceptible, list_infected, list_recovered, list_vaccinated_week, list_newly_infected, list_newly_vaccinated = simulate_pandemic_with_vaccination_challenge(934,0.6, 6, 0.25, 0.6, 1)
fig, ax = plt.subplots(figsize=(10,6))
list_i0 = [1, 1, 3, 5, 9, 17, 32, 32, 17, 5, 2, 1, 0, 0, 0]
print(len(list_i0))
ax.plot(num_weeks, list_i0, label = 'true newly infected')
ax.plot(num_weeks, list_results, label='estimated newly infected')
ax.legend()
ax.grid()
plt.savefig('problem1.5.png', format='PNG')
ax.plot()

In [ ]:
avg_new_infected = []
avg_new_vaccinated = []
avg_susceptible = []
avg_infected = []
avg_recovered = []
avg_vaccinated = []
sum_check = []
for i in range (15):
    avg_new_infected.append(sum(list_newly_infected[i])/len(list_newly_infected[i]))
    if len(list_newly_vaccinated[i])==0:
        den = 1
    else:
        den = len(list_newly_vaccinated[i])
    avg_new_vaccinated.append(sum(list_newly_vaccinated[i])/den)
    avg_susceptible.append(sum(list_susceptible[i])/len(list_susceptible[i]))
    avg_infected.append(sum(list_infected[i])/len(list_infected[i]))
    avg_recovered.append(sum(list_recovered[i])/len(list_recovered[i]))
    avg_vaccinated.append(sum(list_vaccinated_week[i])/len(list_vaccinated_week[i]))
    sum_check.append(sum(list_susceptible[i]+list_infected[i]+list_recovered[i]))

fig, ax = plt.subplots(figsize=(10,6))

ax.plot(num_weeks, avg_susceptible, label = "susceptible")
ax.plot(num_weeks, avg_infected, label = "infected")
ax.plot(num_weeks, avg_recovered, label = "recovered")
ax.plot(num_weeks, avg_vaccinated, label = "vaccinated")
plt.yticks(np.arange(0, 950, 50, dtype=int))
ax.grid()
ax.legend()
plt.savefig('problem1.5_2.png', format='PNG')
plt.show()

# 2. COLORING

## COLORING IN A LINE GRAPH

In [ ]:
G = nx.Graph()
for el in range(10-1):
    G.add_edge(el, el+1)
nx.draw(G, with_labels=True)

In [ ]:
n_players = len(G.nodes)
#we create a vector with the colors of all the nodes, if the node is red:0 and if it is green:1, we set at the beginning all the nodes to 0
list_colors = [0]*n_players
#available actions are assume or remain red:0 and assume or remain green:1
actions = [0,1]
n_actions = len(actions)
#adjacency matrix
W = nx.convert_matrix.to_numpy_matrix(G)

#list of the results of the potential function
list_pot_func = []

convergence = False
t = 0

while convergence == False:

    #choose the node uniformly at random
    moving_player = random.choice(np.array(G.nodes))
    #find the list of its neighbors
    list_neighbors = list(G.neighbors(el))
    #define the inverse noise parameter
    eta = t/100
    #create a vector that have as index the color and as number the number of neighbors of that color
    vec_num_color = [0]*n_actions
    for el in list_neighbors:
        col = list_colors[el]
        vec_num_color[col] += 1

    prob_vect = [0]*n_actions
    #find the number of colors of the neighbors of the node
    for i in actions:
        denominator = [math.exp((-eta)*vec_num_color[i]) for i in range(len(list(vec_num_color)))]
        prob_vect[i] = math.exp((-eta)*vec_num_color[i])/(sum(denominator))

    choosen_action = np.random.choice(np.array(actions), p=prob_vect)
    #update the color for the moving player
    list_colors[moving_player] = choosen_action

    #now calculate the potential function
    counter = 0
    for i in range(len(list_colors)):
        for j in range(1, len(list_colors)):
            if j in list(G.neighbors(i)):
                if list_colors[i]==list_colors[j]:
                    counter += 1
    
    PotFunc = 1/2*counter
    list_pot_func.append(PotFunc)
    
    #if potential function = 0 we have reached convergence
    if PotFunc == 0:
        convergence = True
    else:
        t += 1


In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
list_iterations = [i for i in range(len(list_pot_func))]
ax.plot(list_iterations, list_pot_func)
plt.yticks(np.arange(0, 10, 1, dtype=int))
ax.grid()
plt.savefig('problem2.png', format='PNG')

In [ ]:
print(list_colors)

## COLORING IN A ROUTER NETWORK

In [ ]:
#create a function that given the agent and the adjacency matrix returns the list of neighbors of the agent
def create_list_neighbors(W, node):
    list_neighbors = []
    for i, row in enumerate(W):
        if i == node:
            for i, el in enumerate(row):
                if el == 1:
                    list_neighbors.append(i)

    return list_neighbors


#function that returns the cost of a node to have a certain color
def cost_color(node, color, W, list_colors):
    list_neighbors = create_list_neighbors(W, node)
    counter = 0
    for i in list_neighbors:
        if list_colors[i] == color:
            counter += 2
        elif abs(list_colors[i]-color)==1:
            counter += 1
    
    return counter
    

In [ ]:
#new adjacency matrix
W = np.loadtxt("wifi.mat")
coords = np.loadtxt("coords.mat")

#update the set of available actions:1 : red, 2 : green, 3 : blue, 4 : yellow, 5 : magenta, 6 : cyan, 7 : white, 8 : black
actions = [1, 2, 3, 4, 5, 6, 7, 8]
n_actions = len(actions)
#create an array nodes
nodes = [i for i in range(100)]
n_players = 100
list_colors = [0]*n_players
#list of the results of the potential function
list_pot_func = []
t = 0

while t<5000:
    
    #choose the node uniformly at random
    moving_player = random.choice(nodes)
    #define the inverse noise parameter
    eta = t/100
    list_neighbors = create_list_neighbors(W, moving_player)
  
    prob_vect = [0]*n_actions
    for i in range(n_actions):
        numerator = math.exp((-eta)*cost_color(moving_player, i, W, list_colors))
        denominator = [math.exp((-eta)*cost_color(moving_player, i, W, list_colors)) for i in range(n_actions)]
        prob_vect[i] = numerator/sum(denominator)
    
    choosen_action = np.random.choice(np.array(actions), p=prob_vect)
    #update the color for the moving player
    list_colors[moving_player] = choosen_action

    #now calculate the potential function
    counter = 0
    for i in range(len(list_colors)):
        for j in range(1, len(list_colors)):
            if j in list(create_list_neighbors(W, i)):
                if list_colors[i]==list_colors[j]:
                    counter += 1
    
    PotFunc = 1/2*counter
    list_pot_func.append(PotFunc)
        
    t += 1

In [ ]:
print(len(list_pot_func))
print(list_pot_func[len(list_pot_func)-1])
list_iterations = [i for i in range(len(list_pot_func))]
fig, ax = plt.subplots(figsize=(10,6))
ax.plot(list_iterations, list_pot_func)
plt.yticks(np.arange(0, 130, 10, dtype=int))
ax.grid()
plt.savefig('problem2.2.png', format='PNG')
plt.show()

## COLORING WITH FIXED ETA

In [ ]:
#create an array nodes
nodes = [i for i in range(100)]
n_players = 100
list_colors = [0]*n_players
#list of the results of the potential function
list_pot_func = []
t = 0

while t<1000:
    
    #choose the node uniformly at random
    moving_player = random.choice(nodes)
    #define the inverse noise parameter
    eta = 10
    list_neighbors = create_list_neighbors(W, moving_player)
  
    prob_vect = [0]*n_actions
    for i in range(n_actions):
        numerator = math.exp((-eta)*cost_color(moving_player, i, W, list_colors))
        denominator = [math.exp((-eta)*cost_color(moving_player, i, W, list_colors)) for i in range(n_actions)]
        prob_vect[i] = numerator/sum(denominator)
    
    choosen_action = np.random.choice(np.array(actions), p=prob_vect)
    #update the color for the moving player
    list_colors[moving_player] = choosen_action

    #now calculate the potential function
    counter = 0
    for i in range(len(list_colors)):
        for j in range(1, len(list_colors)):
            if j in list(create_list_neighbors(W, i)):
                if list_colors[i]==list_colors[j]:
                    counter += 1
    
    PotFunc = 1/2*counter
    list_pot_func.append(PotFunc)
        
    t += 1

In [ ]:
print(len(list_pot_func))
print(list_pot_func[len(list_pot_func)-1])
list_iterations = [i for i in range(len(list_pot_func))]
fig, ax = plt.subplots(figsize=(10,6))
ax.plot(list_iterations, list_pot_func)
plt.yticks(np.arange(0, 130, 10, dtype=int))
ax.grid()
#plt.savefig('problem2.3.png', format='PNG')
plt.show()

In [ ]:
#create an array nodes
nodes = [i for i in range(100)]
n_players = 100
list_colors = [0]*n_players
#list of the results of the potential function
list_pot_func = []
t = 0

while t<1000:
    
    #choose the node uniformly at random
    moving_player = random.choice(nodes)
    #define the inverse noise parameter
    eta = 250
    list_neighbors = create_list_neighbors(W, moving_player)
  
    prob_vect = [0]*n_actions
    for i in range(n_actions):
        numerator = math.exp((-eta)*cost_color(moving_player, i, W, list_colors))
        denominator = [math.exp((-eta)*cost_color(moving_player, i, W, list_colors)) for i in range(n_actions)]
        prob_vect[i] = numerator/sum(denominator)
    
    choosen_action = np.random.choice(np.array(actions), p=prob_vect)
    #update the color for the moving player
    list_colors[moving_player] = choosen_action

    #now calculate the potential function
    counter = 0
    for i in range(len(list_colors)):
        for j in range(1, len(list_colors)):
            if j in list(create_list_neighbors(W, i)):
                if list_colors[i]==list_colors[j]:
                    counter += 1
    
    PotFunc = 1/2*counter
    list_pot_func.append(PotFunc)
        
    t += 1

In [ ]:
print(len(list_pot_func))
print(list_pot_func[len(list_pot_func)-1])
list_iterations = [i for i in range(len(list_pot_func))]
fig, ax = plt.subplots(figsize=(10,6))
ax.plot(list_iterations, list_pot_func)
plt.yticks(np.arange(0, 130, 10, dtype=int))
ax.grid()
#plt.savefig('problem2.3.png', format='PNG')
plt.show()

In [ ]:
def apply_coloring_wifi(eta):    
    #create an array nodes
    nodes = [i for i in range(100)]
    n_players = 100
    list_colors = [0]*n_players
    #list of the results of the potential function
    list_pot_func = []
    t = 0

    while t<1000:
        
        #choose the node uniformly at random
        moving_player = random.choice(nodes)
        list_neighbors = create_list_neighbors(W, moving_player)
    
        prob_vect = [0]*n_actions
        for i in range(n_actions):
            numerator = math.exp((-eta)*cost_color(moving_player, i, W, list_colors))
            denominator = [math.exp((-eta)*cost_color(moving_player, i, W, list_colors)) for i in range(n_actions)]
            prob_vect[i] = numerator/sum(denominator)
        
        choosen_action = np.random.choice(np.array(actions), p=prob_vect)
        #update the color for the moving player
        list_colors[moving_player] = choosen_action

        #now calculate the potential function
        counter = 0
        for i in range(len(list_colors)):
            for j in range(1, len(list_colors)):
                if j in list(create_list_neighbors(W, i)):
                    if list_colors[i]==list_colors[j]:
                        counter += 1
        
        PotFunc = 1/2*counter
        list_pot_func.append(PotFunc)
            
        t += 1

    return list_pot_func[len(list_pot_func)-1]

In [ ]:
list_param_eta = [1, 25, 50, 75, 100, 125, 150, 175, 200]
list_pot = []
for i in list_param_eta:
    num = apply_coloring_wifi(i)
    list_pot.append(num)

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
ax.plot(list_param_eta, list_pot)
plt.yticks(np.arange(1, 20, 1, dtype=int))
ax.grid()
plt.savefig('problem2.enne.png', format='PNG')
plt.show()